# Fresh or rotten banana classifer

This is a model that classifies if the bananas are fresh or rotten. <br>
The dataset used is a modified version of this one: "https://www.kaggle.com/datasets/sriramr/fruits-fresh-and-rotten-for-classification".<br> or you could use this: "https://www.kaggle.com/datasets/akash47anand/freshrotten-banana"<br>
Create two folders: dataset(which is further classified into test and train) and Final test

In [3]:
# Code to generate a model
import os
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

train_data_dir = r'dataset\test'
test_data_dir = r'dataset\train'

IMG_WIDTH, IMG_HEIGHT = 100, 100
BATCH_SIZE = 16
NUM_EPOCHS = 5
optimizer = "adam"

# Define classes to include (i.e., folders to consider) in train and test data
included_classes = ['freshbanana', 'rottenbanana']

train_datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)
train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(IMG_WIDTH, IMG_HEIGHT),
    batch_size=BATCH_SIZE,
    class_mode='binary',
    subset='training',
    classes=included_classes
)

validation_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(IMG_WIDTH, IMG_HEIGHT),
    batch_size=BATCH_SIZE,
    class_mode='binary',
    subset='validation',
    classes=included_classes
)

test_datagen = ImageDataGenerator(rescale=1./255)
test_generator = test_datagen.flow_from_directory(
    test_data_dir,
    target_size=(IMG_WIDTH, IMG_HEIGHT),
    batch_size=BATCH_SIZE,
    class_mode='binary',
    classes=included_classes
)

# Create model
model = Sequential()

model.add(Conv2D(16, (3, 3), activation='relu', input_shape=(IMG_WIDTH, IMG_HEIGHT, 3)))
model.add(MaxPooling2D(2, 2))
model.add(Flatten(input_shape=(IMG_WIDTH, IMG_HEIGHT, 3)))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])

# Fit the model
model.fit(train_generator, validation_data=validation_generator, epochs=NUM_EPOCHS)

# Evaluate the model on the test dataset
test_loss, test_accuracy = model.evaluate(test_generator)
print("Test Loss:", test_loss*100,"%")
print("Test Accuracy:", test_accuracy*100,"%")
model.summary()

# Save the trained model
model.save(r'banana_classifier_model.h5')

Found 729 images belonging to 2 classes.
Found 182 images belonging to 2 classes.
Found 3775 images belonging to 2 classes.
Epoch 1/5
46/46 [==============================] - 13s 239ms/step - loss: 0.6537 - accuracy: 0.7682 - val_loss: 0.1287 - val_accuracy: 0.9780
Epoch 2/5
46/46 [==============================] - 10s 212ms/step - loss: 0.1297 - accuracy: 0.9534 - val_loss: 0.0727 - val_accuracy: 0.9835
Epoch 3/5
46/46 [==============================] - 10s 209ms/step - loss: 0.1064 - accuracy: 0.9575 - val_loss: 0.0985 - val_accuracy: 0.9725
Epoch 4/5
46/46 [==============================] - 10s 209ms/step - loss: 0.0807 - accuracy: 0.9684 - val_loss: 0.0715 - val_accuracy: 0.9945
Epoch 5/5
236/236 [==============================] - 29s 123ms/step - loss: 0.0427 - accuracy: 0.9870
Test Loss: 4.272947087883949 %
Test Accuracy: 98.70198965072632 %
Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape           

c:\Users\Akash Anand\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [4]:
# Code to create GUI to test the model
import os
import tkinter as tk
from tkinter import filedialog
from PIL import Image, ImageTk
import tensorflow as tf
from tensorflow.keras.models import load_model

# Load the trained model
model_path = r'banana_classifier_model.h5'
model = load_model(model_path)

def preprocess_image(image_path):
    img = tf.keras.preprocessing.image.load_img(image_path, target_size=(100, 100))
    img_array = tf.keras.preprocessing.image.img_to_array(img)
    img_array = img_array / 255.0
    img_array = img_array.reshape(1, 100, 100, 3)
    return img_array

def predict_banana(img_array):
    prediction = model.predict(img_array)
    if prediction[0][0] < 0.5:
        return "Fresh banana."
    else:
        return "Rotten banana."

def open_image():
    file_path = filedialog.askopenfilename(filetypes=[("Image Files", "*.png;*.jpg;*.jpeg")])
    if file_path:
        img_array = preprocess_image(file_path)
        prediction = predict_banana(img_array)
        img = Image.open(file_path)
        img = img.resize((400, 400), Image.ANTIALIAS)
        img = ImageTk.PhotoImage(img)
        panel.configure(image=img)
        panel.image = img
        result_label.config(text=prediction)

root = tk.Tk()
root.title("Fresh/Rotten Banana Identifier")
root.geometry("450x550")
open_button = tk.Button(root, text="Open Image", command=open_image)
open_button.pack(pady=10)

panel = tk.Label(root, width=400, height=400)
panel.pack()
result_label = tk.Label(root, text="", font=("Helvetica", 16))
result_label.pack(pady=10)

root.mainloop()

1/1 [==============================] - 0s 210ms/step


C:\Users\Akash Anand\AppData\Local\Temp\ipykernel_8528\3469999258.py:33: DeprecationWarning: ANTIALIAS is deprecated and will be removed in Pillow 10 (2023-07-01). Use LANCZOS or Resampling.LANCZOS instead.
  img = img.resize((400, 400), Image.ANTIALIAS)
